In [1]:
import sys
sys.path.insert(0, '/tf/utils/')

In [2]:
from utils import performance
from evaluate import NoisyTargetEvaluator
import tensorflow.keras.backend as K
from keras.utils import custom_object_scope
import tensorflow as tf

In [3]:
base_shape_size = 8192
ws = 255
ol = 128

In [5]:
def weighted_msle(y_true, y_pred):
    # Define o peso para penalizar mais as subestimações
    w = 2.0

    # Evita valores negativos adicionando 1 antes de aplicar o log
    log_true = 20 * tf.math.log(y_true + 1)
    log_pred = 20 * tf.math.log(y_pred + 1)

    # Cálculo do erro
    error = log_true - log_pred

    # Aplica pesos diferentes para superestimação e subestimação
    weighted_error = tf.where(error > 0, w * error, error)

    # Retorna a média do erro logarítmico quadrado ponderado
    return tf.reduce_mean(tf.abs(weighted_error))

In [6]:
with custom_object_scope({'weighted_msle': weighted_msle}):
    eval = NoisyTargetEvaluator(
                base_shape_size,
                '/tf/Dados/Vozes/test/',
                '/tf/Dados/Ruido/test/',
                './models/att-res-unet-loss-0.5484-epochs-41-2023-11-23_12-17-40.h5',
                'Attention Residual U-Net'
            )

Loading Speech Files: 100%|██████████| 819/819 [00:01<00:00, 452.41it/s]



In [8]:
df_resultado = eval.evaluate(batch_num=50, module_only=True)

 52%|█████▏    | 26/50 [03:01<02:47,  6.96s/it]/usr/local/lib/python3.8/dist-packages/pystoi/stoi.py:66: RuntimeWarning: Not enough STFT frames to compute intermediate intelligibility measure after removing silent frames. Returning 1e-5. Please check you wav files
  warnings.warn('Not enough STFT frames to compute intermediate '
100%|██████████| 50/50 [05:52<00:00,  7.05s/it]


File saved to Attention Residual U-Net-metrics_2023-11-29_00-23-50.xlsx


In [9]:
df_resultado.describe()

,PESQ,STOI,SNR,PESQ (Filtered),STOI (Filtered),SNR (Filtered)
count,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000,6386.000000
mean,2.128068,0.847709,9.321164,3.174915,0.896044,11.515781
std,0.555968,0.132823,5.593846,0.645557,0.104411,4.429138
min,1.052631,0.000010,-0.110167,1.040000,0.000010,-1.774675
25%,1.697100,0.782099,4.842880,2.722067,0.863119,7.973616
50%,2.061256,0.886126,9.634803,3.251435,0.929969,11.148349
75%,2.467189,0.948356,13.997268,3.693687,0.965991,14.675514
max,4.443657,0.999070,29.110978,4.373991,0.998661,29.469428
